#### Reading data from csv

In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel('Residual.xlsx',encoding='mac_roman')
#test_df = pd.read_csv('UNSPSCtestDataSet.csv',encoding='mac_roman',low_memory=False)
# y and X are labels and features for the training set respectively
#df = df.drop_duplicates(['PO_Line_Description'])
#test_df = test_df.drop_duplicates(['MaterialDescription'])
y = df['UNSPSC_Code']
X = df['PO_Line_Description']
X.head()

0               FIN-0901MONTHLY FILLING OF 2015.8 VAT
1                FIN-08007 MONTHLY FILLING 2015.7 VAT
2                                      VAT SALES ELCA
3            PROJECT GO- BWI- CONTRACT PO - LEASEHOLD
4    DIR.BUNDESSTEUER 01.07.13-30.06.14/KANTON SCHWYZ
Name: PO_Line_Description, dtype: object

In [2]:
def description_to_words(sentence):
    # Keeping only text
    import re
    from nltk.corpus import stopwords
    text = re.sub("[^a-zA-Z]"," ",sentence) #[^a-zA-Z0-9] REMOVING NUMBER FROM THIS
    # Convert words to lower case and split them 
    text = text.lower().split(" ")
    # optionally remove stopwords
    stops = set(stopwords.words('english'))
    meaningful_words = [w for w in text if not w in stops]
    # Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

def sentences_split(field):
    from nltk.corpus import stopwords
    # Tokenizing each sentence from the paragraph 
    #raw_sentences = tokenizer.tokenize(word.strip())
    
    # Sentence list 
    import re
    field = re.sub("[^a-zA-Z]"," ",str(field))
    sentences = []
    if len(field) > 0:
        sentences.append(description_to_words(field))
    return sentences

def list_of_words(series):
    mainsentences = []
    for i in series:
       mainsentences += sentences_split(i)
    x = get_features(mainsentences)
    return x
    

# BAG OF WORDS MODEL 
def get_features(series):
    from sklearn.feature_extraction.text import CountVectorizer

    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 5000) 
    train_data_features = vectorizer.fit_transform(series)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()
    return train_data_features


### Multilayer perceptron model in Tensorflow

In [3]:
### Divdindg total data into batches

def batches(k_batches, features, labels): 
    assert len(features) == len(labels)
    batches_total = []
    total_size = len(features)
    for start_i in range(0,total_size, k_batches):
        end_i = start_i + k_batches
        batch = [features[start_i : end_i], labels[start_i : end_i]]
        batches_total.append(batch)
    return batches_total

In [48]:
int(len(X)-len(X)/3) - int(len(X)-len(X)/3)

276627

In [51]:
x = len(X)/8 
x 

51867.625

In [8]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
        C:\Users\sunka\AppData\Roaming\nltk_data...
      Unzipping corpora\stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [4]:
import tensorflow as tf


train_features = list_of_words(X[:int(len(X)-len(X)/3)])
train_labels = y[:int(len(X)-len(X)/3)]

val_features = list_of_words(X[int(len(X)-len(X)/3):int(len(X)-len(X)/3+len(X)/8)])
val_labels = y[int(len(X)-len(X)/3):int(len(X)-len(X)/3+len(X)/8)]
test_features = list_of_words(X[int(len(X)-len(X)/3+len(X)/8):])
test_labels = y[int(len(X)-len(X)/3+len(X)/8):]

print(train_features.shape)
print(val_features.shape)


(276627, 5000)
(51867, 5000)


In [54]:
unique_train_labels = y.unique()
#print(len(unique_train_labels)) 1161

#print(len(test_labels.unique())) 663

bigdata = pd.concat([y,test_labels], ignore_index=True)
print(len(y.unique()))

91


In [ ]:
import time
start = time.time()
n_inputs = 5000
n_classes = 91
features = tf.placeholder(tf.float32, [None, n_inputs])
labels = tf.placeholder(tf.int32, [None])

h_layer_1 = 2900
h_layer_2 = 2000

weights = {
'hidden_weights1' : tf.Variable(tf.truncated_normal([n_inputs, h_layer_1])),
'hidden_weights2' : tf.Variable(tf.truncated_normal([h_layer_1, h_layer_2])),
'out_weights' : tf.Variable(tf.truncated_normal([h_layer_2, n_classes]))
}

bias = {
'hidden_bias1' : tf.Variable(tf.truncated_normal([h_layer_1])),
'hidden_bias2' : tf.Variable(tf.truncated_normal([h_layer_2])),
'out_bias' : tf.Variable(tf.truncated_normal([n_classes]))
}
keep_prob = tf.placeholder(tf.float32)

### Hidden layer 1 
hidden_output1 = tf.add(tf.matmul(features, weights['hidden_weights1']),bias['hidden_bias1'])
hidden_relu1 = tf.nn.relu(hidden_output1)
hidden_out1 = tf.nn.dropout(hidden_relu1, keep_prob)

### Hidden layer 2
hidden_output2 = tf.add(tf.matmul(hidden_out1, weights['hidden_weights2']),bias['hidden_bias2'])
hidden_relu2 = tf.nn.relu(hidden_output2)
hidden_out2 = tf.nn.dropout(hidden_relu2, keep_prob)

### Output layer 
logits = tf.add(tf.matmul(hidden_out2, weights['out_weights']),bias['out_bias'])
logits =  tf.nn.relu(logits)
learn_rate = 0.7 # 0.5 + 200 with a dropout in output layer + 0.7prob for training set= 99.6%

# The file path to save the data
save_file = './train_model.ckpt'
saver = tf.train.Saver() # Class to save the model


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf.one_hot(labels, n_classes)))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learn_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(tf.one_hot(labels, n_classes),1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

batchSize =  10 

epochs = 1000 #  200epoch+.5lr = 99.6
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    total_batches = batches(batchSize, train_features, train_labels)
    
    for epoch in range(epochs): 
        for batch_features, batch_labels in total_batches: 
            train_data = {features: batch_features, labels : batch_labels, keep_prob : 0.5}
            sess.run(optimizer, feed_dict = train_data)
        # Print status for every 100 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: val_features,
                    labels: val_labels,
                    keep_prob : 0.5})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))
    Accuracy = sess.run(accuracy, feed_dict={features : test_features, labels :test_labels, keep_prob : 1.0})
    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')
print("Accuracy value is {}".format(Accuracy))
end = time.time()
print("total time ", end-start)

#### Restoring the saved model 


In [ ]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(accuracy, feed_dict={features : test_features, labels :test_labels, keep_prob : 1.0})

print('Test Accuracy: {}'.format(test_accuracy))